In [11]:
import pandas as pd
import h3
import random
import os

In [12]:
# Constants
NUM_POINTS = 1000  # Number of points
RESOLUTION = 9  # H3 resolution level
SOUTH_HOLLAND_BOUNDS = {
    "min_lat": 51.4343,
    "max_lat": 52.3890,
    "min_lon": 3.8500,
    "max_lon": 4.8952
}

In [13]:
def generate_random_location(bounds):
    """
    Generate a random location within the specified bounds.
    
    Parameters:
    bounds (dict): A dictionary with the keys 'min_lat', 'max_lat', 'min_lon', 'max_lon'.
    
    Returns:
    tuple: A tuple containing the latitude and longitude.
    """
    lat = random.uniform(bounds["min_lat"], bounds["max_lat"])
    lon = random.uniform(bounds["min_lon"], bounds["max_lon"])
    return lat, lon

In [14]:
# Generate mock data for H3 hexagons
data = [{
    "hex_address": h3.geo_to_h3(*generate_random_location(SOUTH_HOLLAND_BOUNDS), RESOLUTION),
    "latitude": lat,
    "longitude": lon,
    "value": random.randint(1, 1000)  # Random value between 1 and 1000
} for lat, lon in (generate_random_location(SOUTH_HOLLAND_BOUNDS) for _ in range(NUM_POINTS))]

In [15]:
# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
csv_path = "hex/h3_farm_mock_data.csv"

# Create directory if it doesn't exist
if not os.path.exists('hex'):
    os.makedirs('hex')

# Now you can save the file
df.to_csv(csv_path, index=False)